# Collect all relevant files

This notebook aims at recovering all the files needed for today's notebooks:
- fill BUCKET variable with th ename of your bucket
- check BQ_DATASET for your dataset name
- check if REGION is the one you need

In [1]:
import os
# use here the name of your own bucket and bigquery dataset
BUCKET = 'telemar-formazione-master-day6'
BQ_DATASET = 'telemar_formazione_master_day6'

os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = 'europe-west1'
os.environ['BQ_DATASET'] = BQ_DATASET

In [2]:
%%bash
echo "create bucket gs://$BUCKET and copy data into"
gsutil mb -p $PROJECT_ID -c regional -l $REGION  gs://$BUCKET/ 

echo "data copy"

FROM=gs://data-science-on-gcp/flights/chapter8/output
TO=gs://$BUCKET/flights/chapter8/output

CMD="gsutil -m cp "
for SHARD in `seq -w 0 6`; do
  CMD="$CMD ${FROM}/testFlights-0000${SHARD}-of-00007.csv"
done
for SHARD in `seq -w 0 6`; do
  CMD="$CMD ${FROM}/trainFlights-0000${SHARD}-of-00007.csv"
done
CMD="$CMD $TO"

$CMD

create bucket gs://telemar-formazione-master-day6 and copy data into
data copy


Creating gs://telemar-formazione-master-day6/...
Copying gs://data-science-on-gcp/flights/chapter8/output/testFlights-00000-of-00007.csv [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/chapter8/output/testFlights-00001-of-00007.csv [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/chapter8/output/testFlights-00002-of-00007.csv [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/chapter8/output/testFlights-00003-of-00007.csv [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/chapter8/output/testFlights-00004-of-00007.csv [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/chapter8/output/testFlights-00005-of-00007.csv [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/chapter8/output/testFlights-00006-of-00007.csv [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/chapter8/output/trainFlights-00000-of-00007.csv [Content-Type=text/plain]...
Copying gs://d

In [3]:
%%bash

FROM=gs://data-science-on-gcp/flights/tzcorr
TO=gs://$BUCKET/flights/tzcorr

#sharded files
CMD="gsutil -m cp "
for SHARD in `seq -w 0 35`; do
  CMD="$CMD ${FROM}/all_flights-000${SHARD}-of-00036"
done
CMD="$CMD $TO"
echo $CMD
$CMD

# individual files
CMD="gsutil -m cp "
for FILE in airports/airports.csv.gz; do
   CMD="$CMD gs://data-science-on-gcp/flights/$FILE gs://$BUCKET/flights/$FILE"
done
echo $CMD
$CMD

gsutil -m cp gs://data-science-on-gcp/flights/tzcorr/all_flights-00000-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00001-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00002-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00003-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00004-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00005-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00006-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00007-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00008-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00009-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00010-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00011-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00012-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_flights-00013-of-00036 gs://data-science-on-gcp/flights/tzcorr/all_fligh

Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00000-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00001-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00002-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00003-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00004-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00005-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00006-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00007-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-00008-of-00036 [Content-Type=text/plain]...
Copying gs://data-science-on-gcp/flights/tzcorr/all_flights-0000

In [4]:
%%bash

bq mk $BQ_DATASET

ONE_FILE=$(gsutil ls gs://${BUCKET}/flights/tzcorr/all_flights-00001*)
echo "Creating table definition from $ONE_FILE"
bq mk --external_table_definition=./tzcorr.json@CSV=$ONE_FILE ${BQ_DATASET}.fedtzcorr

echo "Loading all files into BigQuery"
bq load ${BQ_DATASET}.tzcorr "gs://${BUCKET}/flights/tzcorr/all_flights-*" tzcorr.json

Dataset 'telemar-formazione-master:telemar_formazione_master_day6' successfully created.
Creating table definition from gs://telemar-formazione-master-day6/flights/tzcorr/all_flights-00001-of-00036
Table 'telemar-formazione-master:telemar_formazione_master_day6.fedtzcorr' successfully created.
Loading all files into BigQuery



Waiting on bqjob_r4cf034b89062e68f_00000164a798e427_1 ... (66s) Current status: DONE   